In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Activation, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from keras.layers import Conv1D, MaxPooling1D
import numpy as np

Using TensorFlow backend.


In [2]:
df = pd.read_csv('energy_nw_clean_dataframe.csv')

In [3]:
df.head()

AR_NUMBER AR_PRIORITY AR_SEVERITY PRIORITY_SEVERITY  \
0     383472         CAQ           C             CAQ:C   
1     383473         CAQ           D             CAQ:D   
2     383474         CAQ           D             CAQ:D   
3     383476        NCAQ           4            NCAQ:4   
4     383477         CAQ           D             CAQ:D   

                                 CONCAT_TEXT_FOR_WKS  
0  rcic hpcs low cst level swap fill vent suction...  
1  rcic p not start run sop rcic fill direct star...  
2  hp drop hpcs suction switchover alarm hp drop ...  
3                   bre bre need window evaluate bre  
4  receive rod drive control sys inop alarm recei...

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11337 entries, 0 to 11336
Data columns (total 5 columns):
AR_NUMBER              11337 non-null int64
AR_PRIORITY            11337 non-null object
AR_SEVERITY            11337 non-null object
PRIORITY_SEVERITY      11337 non-null object
CONCAT_TEXT_FOR_WKS    11337 non-null object
dtypes: int64(1), object(4)
memory usage: 443.0+ KB


In [5]:
df.PRIORITY_SEVERITY.value_counts()

NCAQ:4    4423
NCAQ:3    3755
CAQ:D     1982
CAQ:C     1098
CAQ:B       55
NCAQ:2      22
CAQ:A        2
Name: PRIORITY_SEVERITY, dtype: int64

In [6]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['CONCAT_TEXT_FOR_WKS'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 18884 unique tokens.


In [7]:
X = tokenizer.texts_to_sequences(df['CONCAT_TEXT_FOR_WKS'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (11337, 250)


In [8]:
Y = pd.get_dummies(df['PRIORITY_SEVERITY']).values
print('Shape of data tensor:', Y.shape)

Shape of data tensor: (11337, 7)


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(10203, 250) (10203, 7)
(1134, 250) (1134, 7)


In [10]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy', 'accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 707       
Total params: 5,081,107
Trainable params: 5,081,107
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
#This isn't important right now, it printouts look weird and i'm not sure what happened here.
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.1)

Train on 9182 samples, validate on 1021 samples
Epoch 1/5


7424/9182 [=======================>......] - ETA: 4:52 - loss: 0.6930 - categorical_accuracy: 0.1250 - acc: 0.49 - ETA: 2:53 - loss: 0.6919 - categorical_accuracy: 0.1406 - acc: 0.56 - ETA: 2:12 - loss: 0.6907 - categorical_accuracy: 0.1458 - acc: 0.62 - ETA: 1:52 - loss: 0.6895 - categorical_accuracy: 0.1484 - acc: 0.66 - ETA: 1:41 - loss: 0.6881 - categorical_accuracy: 0.1688 - acc: 0.70 - ETA: 1:33 - loss: 0.6864 - categorical_accuracy: 0.1693 - acc: 0.72 - ETA: 1:26 - loss: 0.6845 - categorical_accuracy: 0.1942 - acc: 0.74 - ETA: 1:22 - loss: 0.6823 - categorical_accuracy: 0.2070 - acc: 0.76 - ETA: 1:18 - loss: 0.6798 - categorical_accuracy: 0.2153 - acc: 0.77 - ETA: 1:15 - loss: 0.6765 - categorical_accuracy: 0.2313 - acc: 0.78 - ETA: 1:12 - loss: 0.6720 - categorical_accuracy: 0.2429 - acc: 0.78 - ETA: 1:10 - loss: 0.6661 - categorical_accuracy: 0.2487 - acc: 0.79 - ETA: 1:09 - loss: 0.6568 - categorical_accuracy: 0.2560 - acc: 0.79 - ETA: 1:07 - loss: 0.6441 - categorical_accura

9182/9182 [==============================] - ETA: 11s - loss: 0.3507 - categorical_accuracy: 0.3726 - acc: 0.850 - ETA: 10s - loss: 0.3503 - categorical_accuracy: 0.3729 - acc: 0.850 - ETA: 10s - loss: 0.3499 - categorical_accuracy: 0.3732 - acc: 0.850 - ETA: 10s - loss: 0.3496 - categorical_accuracy: 0.3733 - acc: 0.850 - ETA: 9s - loss: 0.3493 - categorical_accuracy: 0.3733 - acc: 0.850 - ETA: 9s - loss: 0.3489 - categorical_accuracy: 0.3733 - acc: 0.85 - ETA: 8s - loss: 0.3487 - categorical_accuracy: 0.3718 - acc: 0.85 - ETA: 8s - loss: 0.3483 - categorical_accuracy: 0.3724 - acc: 0.85 - ETA: 7s - loss: 0.3479 - categorical_accuracy: 0.3733 - acc: 0.85 - ETA: 7s - loss: 0.3474 - categorical_accuracy: 0.3738 - acc: 0.85 - ETA: 7s - loss: 0.3471 - categorical_accuracy: 0.3745 - acc: 0.85 - ETA: 6s - loss: 0.3468 - categorical_accuracy: 0.3740 - acc: 0.85 - ETA: 6s - loss: 0.3466 - categorical_accuracy: 0.3745 - acc: 0.85 - ETA: 5s - loss: 0.3463 - categorical_accuracy: 0.3745 - acc: 0

7488/9182 [=======================>......] - ETA: 1:02 - loss: 0.2897 - categorical_accuracy: 0.4688 - acc: 0.85 - ETA: 1:01 - loss: 0.2972 - categorical_accuracy: 0.4375 - acc: 0.85 - ETA: 1:00 - loss: 0.2902 - categorical_accuracy: 0.4688 - acc: 0.85 - ETA: 1:00 - loss: 0.2889 - categorical_accuracy: 0.4688 - acc: 0.85 - ETA: 1:00 - loss: 0.2914 - categorical_accuracy: 0.4375 - acc: 0.85 - ETA: 1:00 - loss: 0.2903 - categorical_accuracy: 0.4349 - acc: 0.85 - ETA: 59s - loss: 0.2896 - categorical_accuracy: 0.4330 - acc: 0.8565 - ETA: 59s - loss: 0.2891 - categorical_accuracy: 0.4336 - acc: 0.856 - ETA: 59s - loss: 0.2884 - categorical_accuracy: 0.4271 - acc: 0.856 - ETA: 59s - loss: 0.2878 - categorical_accuracy: 0.4375 - acc: 0.856 - ETA: 58s - loss: 0.2902 - categorical_accuracy: 0.4361 - acc: 0.857 - ETA: 57s - loss: 0.2926 - categorical_accuracy: 0.4284 - acc: 0.857 - ETA: 57s - loss: 0.2928 - categorical_accuracy: 0.4243 - acc: 0.857 - ETA: 57s - loss: 0.2944 - categorical_accura

9182/9182 [==============================] - ETA: 11s - loss: 0.2810 - categorical_accuracy: 0.5056 - acc: 0.867 - ETA: 10s - loss: 0.2812 - categorical_accuracy: 0.5051 - acc: 0.867 - ETA: 10s - loss: 0.2809 - categorical_accuracy: 0.5056 - acc: 0.867 - ETA: 10s - loss: 0.2806 - categorical_accuracy: 0.5066 - acc: 0.867 - ETA: 9s - loss: 0.2806 - categorical_accuracy: 0.5068 - acc: 0.868 - ETA: 9s - loss: 0.2806 - categorical_accuracy: 0.5072 - acc: 0.86 - ETA: 8s - loss: 0.2806 - categorical_accuracy: 0.5072 - acc: 0.86 - ETA: 8s - loss: 0.2805 - categorical_accuracy: 0.5072 - acc: 0.86 - ETA: 7s - loss: 0.2802 - categorical_accuracy: 0.5083 - acc: 0.86 - ETA: 7s - loss: 0.2802 - categorical_accuracy: 0.5087 - acc: 0.86 - ETA: 6s - loss: 0.2798 - categorical_accuracy: 0.5098 - acc: 0.86 - ETA: 6s - loss: 0.2796 - categorical_accuracy: 0.5104 - acc: 0.86 - ETA: 6s - loss: 0.2796 - categorical_accuracy: 0.5103 - acc: 0.86 - ETA: 5s - loss: 0.2793 - categorical_accuracy: 0.5107 - acc: 0

...

9182/9182 [==============================] - ETA: 11s - loss: 0.2403 - categorical_accuracy: 0.6095 - acc: 0.892 - ETA: 10s - loss: 0.2402 - categorical_accuracy: 0.6098 - acc: 0.893 - ETA: 10s - loss: 0.2402 - categorical_accuracy: 0.6099 - acc: 0.893 - ETA: 9s - loss: 0.2400 - categorical_accuracy: 0.6107 - acc: 0.893 - ETA: 9s - loss: 0.2399 - categorical_accuracy: 0.6107 - acc: 0.89 - ETA: 9s - loss: 0.2403 - categorical_accuracy: 0.6100 - acc: 0.89 - ETA: 8s - loss: 0.2403 - categorical_accuracy: 0.6099 - acc: 0.89 - ETA: 8s - loss: 0.2399 - categorical_accuracy: 0.6106 - acc: 0.89 - ETA: 7s - loss: 0.2396 - categorical_accuracy: 0.6114 - acc: 0.89 - ETA: 7s - loss: 0.2396 - categorical_accuracy: 0.6113 - acc: 0.89 - ETA: 6s - loss: 0.2394 - categorical_accuracy: 0.6116 - acc: 0.89 - ETA: 6s - loss: 0.2391 - categorical_accuracy: 0.6122 - acc: 0.89 - ETA: 5s - loss: 0.2388 - categorical_accuracy: 0.6126 - acc: 0.89 - ETA: 5s - loss: 0.2388 - categorical_accuracy: 0.6130 - acc: 0.8

7488/9182 [=======================>......] - ETA: 1:03 - loss: 0.1848 - categorical_accuracy: 0.7188 - acc: 0.92 - ETA: 1:02 - loss: 0.2091 - categorical_accuracy: 0.6875 - acc: 0.90 - ETA: 1:01 - loss: 0.2173 - categorical_accuracy: 0.6719 - acc: 0.90 - ETA: 1:01 - loss: 0.2106 - categorical_accuracy: 0.6836 - acc: 0.91 - ETA: 1:01 - loss: 0.2135 - categorical_accuracy: 0.6781 - acc: 0.91 - ETA: 1:01 - loss: 0.2191 - categorical_accuracy: 0.6641 - acc: 0.90 - ETA: 1:00 - loss: 0.2142 - categorical_accuracy: 0.6763 - acc: 0.91 - ETA: 59s - loss: 0.2125 - categorical_accuracy: 0.6875 - acc: 0.9124 - ETA: 59s - loss: 0.2117 - categorical_accuracy: 0.6910 - acc: 0.911 - ETA: 59s - loss: 0.2092 - categorical_accuracy: 0.6937 - acc: 0.912 - ETA: 58s - loss: 0.2085 - categorical_accuracy: 0.6918 - acc: 0.912 - ETA: 58s - loss: 0.2086 - categorical_accuracy: 0.6940 - acc: 0.913 - ETA: 57s - loss: 0.2066 - categorical_accuracy: 0.7031 - acc: 0.915 - ETA: 57s - loss: 0.2072 - categorical_accura

9182/9182 [==============================] - ETA: 11s - loss: 0.2027 - categorical_accuracy: 0.6962 - acc: 0.916 - ETA: 10s - loss: 0.2024 - categorical_accuracy: 0.6964 - acc: 0.916 - ETA: 10s - loss: 0.2023 - categorical_accuracy: 0.6966 - acc: 0.916 - ETA: 9s - loss: 0.2021 - categorical_accuracy: 0.6969 - acc: 0.916 - ETA: 9s - loss: 0.2024 - categorical_accuracy: 0.6967 - acc: 0.91 - ETA: 9s - loss: 0.2021 - categorical_accuracy: 0.6974 - acc: 0.91 - ETA: 8s - loss: 0.2021 - categorical_accuracy: 0.6975 - acc: 0.91 - ETA: 8s - loss: 0.2023 - categorical_accuracy: 0.6973 - acc: 0.91 - ETA: 7s - loss: 0.2024 - categorical_accuracy: 0.6972 - acc: 0.91 - ETA: 7s - loss: 0.2024 - categorical_accuracy: 0.6972 - acc: 0.91 - ETA: 6s - loss: 0.2023 - categorical_accuracy: 0.6974 - acc: 0.91 - ETA: 6s - loss: 0.2023 - categorical_accuracy: 0.6969 - acc: 0.91 - ETA: 5s - loss: 0.2024 - categorical_accuracy: 0.6969 - acc: 0.91 - ETA: 5s - loss: 0.2022 - categorical_accuracy: 0.6975 - acc: 0.9

7488/9182 [=======================>......] - ETA: 1:03 - loss: 0.1910 - categorical_accuracy: 0.7656 - acc: 0.92 - ETA: 1:02 - loss: 0.1642 - categorical_accuracy: 0.8125 - acc: 0.94 - ETA: 1:01 - loss: 0.1673 - categorical_accuracy: 0.7917 - acc: 0.94 - ETA: 1:01 - loss: 0.1620 - categorical_accuracy: 0.8008 - acc: 0.94 - ETA: 1:01 - loss: 0.1702 - categorical_accuracy: 0.7719 - acc: 0.93 - ETA: 1:01 - loss: 0.1703 - categorical_accuracy: 0.7708 - acc: 0.93 - ETA: 1:00 - loss: 0.1725 - categorical_accuracy: 0.7612 - acc: 0.93 - ETA: 59s - loss: 0.1754 - categorical_accuracy: 0.7578 - acc: 0.9328 - ETA: 59s - loss: 0.1725 - categorical_accuracy: 0.7604 - acc: 0.933 - ETA: 59s - loss: 0.1714 - categorical_accuracy: 0.7594 - acc: 0.932 - ETA: 58s - loss: 0.1693 - categorical_accuracy: 0.7642 - acc: 0.933 - ETA: 58s - loss: 0.1716 - categorical_accuracy: 0.7565 - acc: 0.932 - ETA: 57s - loss: 0.1725 - categorical_accuracy: 0.7548 - acc: 0.932 - ETA: 57s - loss: 0.1724 - categorical_accura

9182/9182 [==============================] - ETA: 11s - loss: 0.1721 - categorical_accuracy: 0.7542 - acc: 0.931 - ETA: 10s - loss: 0.1720 - categorical_accuracy: 0.7543 - acc: 0.931 - ETA: 10s - loss: 0.1720 - categorical_accuracy: 0.7543 - acc: 0.931 - ETA: 9s - loss: 0.1717 - categorical_accuracy: 0.7550 - acc: 0.931 - ETA: 9s - loss: 0.1718 - categorical_accuracy: 0.7549 - acc: 0.93 - ETA: 9s - loss: 0.1716 - categorical_accuracy: 0.7555 - acc: 0.93 - ETA: 8s - loss: 0.1718 - categorical_accuracy: 0.7548 - acc: 0.93 - ETA: 8s - loss: 0.1718 - categorical_accuracy: 0.7549 - acc: 0.93 - ETA: 7s - loss: 0.1716 - categorical_accuracy: 0.7557 - acc: 0.93 - ETA: 7s - loss: 0.1716 - categorical_accuracy: 0.7558 - acc: 0.93 - ETA: 6s - loss: 0.1715 - categorical_accuracy: 0.7559 - acc: 0.93 - ETA: 6s - loss: 0.1712 - categorical_accuracy: 0.7562 - acc: 0.93 - ETA: 5s - loss: 0.1712 - categorical_accuracy: 0.7559 - acc: 0.93 - ETA: 5s - loss: 0.1708 - categorical_accuracy: 0.7568 - acc: 0.9